In [1]:
import sqlite3 as lite
import pandas as pd

In [2]:
db = lite.connect('./products.db')

In [4]:
query = """
SELECT *
FROM Orders;
"""

pd.read_sql(query,db).head()

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
0,10248,90,5,1996-07-04,3
1,10249,81,6,1996-07-05,1
2,10250,34,4,1996-07-08,2
3,10251,84,3,1996-07-08,1
4,10252,76,4,1996-07-09,2


In [12]:
query = """
SELECT sql
FROM sqlite_master
;
"""
pd.read_sql(query, db)

,sql
0,"CREATE TABLE Categories(\n ""CategoryID"" TEXT,\n ""CategoryName"" TEXT,\n ""Description"" TEXT\n)"
1,"CREATE TABLE Customers(\n ""CustomerID"" TEXT,\n ""CustomerName"" TEXT,\n ""ContactName"" TEXT,\n ""Address"" TEXT,\n ""City"" TEXT,\n ""PostalCode"" TEXT,\n ""Country"" TEXT\n)"
2,"CREATE TABLE Employees(\n ""EmployeeID"" TEXT,\n ""LastName"" TEXT,\n ""FirstName"" TEXT,\n ""BirthDate"" TEXT,\n ""Photo"" TEXT,\n ""Notes"" TEXT\n)"
3,"CREATE TABLE Shippers(\n ""ShipperID"" TEXT,\n ""ShipperName"" TEXT,\n ""Phone"" TEXT\n)"
4,"CREATE TABLE Suppliers(\n ""SupplierID"" TEXT,\n ""SupplierName"" TEXT,\n ""ContactName"" TEXT,\n ""Address"" TEXT,\n ""City"" TEXT,\n ""PostalCode"" TEXT,\n ""Country"" TEXT,\n ""Phone"" TEXT\n)"
5,"CREATE TABLE Orders(\nOrderID integer,\nCustomerID integer,\nEmployeeID integer,\nOrderDate date,\nShipperID integer)"
6,"CREATE TABLE OrderDetails(\nOrderDetailID integer,\nOrderID integer,\nProductID integer,\nQuantity integer)"
7,"CREATE TABLE Products(\nProductID integer,\nProductName text,\nSupplierID integer,\nCategoryID integer,\nUnit text,\nPrice integer\n)"


In [95]:
#Orders
# OrderID => OrderDetails
# CustomerID => Customers
# EmployeeID => Employees
# ShipperID => Shippers

query = """
SELECT * 
FROM 
    Orders O
    JOIN
        OrderDetails D
    ON
        O.OrderID = D.OrderID
    JOIN
        Customers C
    ON
        O.CustomerID = C.CustomerID
    JOIN
        Employees E
    ON
        O.EmployeeID = E.EmployeeID
    JOIN
        Shippers S
    ON
        O.ShipperID = S.ShipperID
    
;
"""
len(pd.read_sql(query,db))

518

In [11]:
pd.options.display.max_colwidth = 200

In [19]:
query = """
SELECT *
FROM OrderDetails
WHERE
    Quantity >=40
;
"""
pd.read_sql(query,db).head()

,OrderDetailID,OrderID,ProductID,Quantity
0,5,10249,51,40
1,12,10252,20,40
2,14,10252,60,40
3,16,10253,39,42
4,17,10253,49,40


In [17]:
query="""
SELECT *
FROM Orders
;
"""
pd.read_sql(query,db).head()

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
0,10248,90,5,1996-07-04,3
1,10249,81,6,1996-07-05,1
2,10250,34,4,1996-07-08,2
3,10251,84,3,1996-07-08,1
4,10252,76,4,1996-07-09,2


In [21]:
query = """
SELECT *
FROM Orders
WHERE
    OrderID IN (
        SELECT OrderID
        FROM OrderDetails
        WHERE
            Quantity >=40
    )
;
"""
pd.read_sql(query,db).head()

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
0,10249,81,6,1996-07-05,1
1,10252,76,4,1996-07-09,2
2,10253,34,3,1996-07-10,2
3,10258,20,1,1996-07-17,1
4,10260,55,4,1996-07-19,1


In [23]:
query = """
SELECT *
FROM
    Customers
;
"""
pd.read_sql(query,db).head()

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209,Germany
1,2,Ana Trujillo Emparedados y helados,Ana Trujillo,Avda. de la Constitución 2222,México D.F.,05021,Mexico
2,3,Antonio Moreno Taquería,Antonio Moreno,Mataderos 2312,México D.F.,05023,Mexico
3,4,Around the Horn,Thomas Hardy,120 Hanover Sq.,London,WA1 1DP,UK
4,5,Berglunds snabbköp,Christina Berglund,Berguvsvägen 8,Luleå,S-958 22,Sweden


In [27]:
query = """
SELECT CustomerID
FROM
    Customers
WHERE
    CustomerName LIKE '%Ernst%'
    OR
    CustomerName LIKE '%Stop%'
;
"""
pd.read_sql(query,db).head()

,CustomerID
0,20
1,45
2,63


In [30]:
query = """
SELECT *
FROM 
    Orders
WHERE
    OrderID IN (
        SELECT OrderID
        FROM 
            OrderDetails
        WHERE
            Quantity >=40
    )
    AND
    CustomerID IN(
        SELECT CustomerID
        FROM
            Customers
        WHERE
            CustomerName LIKE '%Ernst%'
            OR
            CustomerName LIKE '%Stop%'
    )
    
;
"""
pd.read_sql(query,db).head()
# End of Subject

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
0,10258,20,1,1996-07-17,1
1,10263,20,9,1996-07-23,3
2,10273,63,3,1996-08-05,3
3,10285,63,1,1996-08-20,2
4,10286,63,8,1996-08-21,3


In [32]:
query = """
SELECT *
FROM Products
;
"""
products_df = pd.read_sql(query,db)

In [38]:
products_df.sort_values('ProductName').head()

,ProductID,ProductName,SupplierID,CategoryID,Unit,Price
16,17,Alice Mutton,7,6,20 - 1 kg tins,39.0
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.0
39,40,Boston Crab Meat,19,8,24 - 4 oz tins,18.4
59,60,Camembert Pierrot,28,4,15 - 300 g rounds,34.0
17,18,Carnarvon Tigers,7,8,16 kg pkg.,62.5


In [37]:
products_df.sort_values('ProductName', ascending=False).head()

,ProductID,ProductName,SupplierID,CategoryID,Unit,Price
46,47,Zaanse koeken,22,3,10 - 4 oz boxes,9.50
63,64,Wimmers gute Semmelknödel,12,5,20 bags x 4 pieces,33.25
62,63,Vegie-spread,7,2,15 - 625 g jars,43.90
49,50,Valkoinen suklaa,23,3,12 - 100 g bars,16.25
6,7,Uncle Bob's Organic Dried Pears,3,7,12 - 1 lb pkgs.,30.00


In [43]:
query = """
SELECT *
FROM Products
    ORDER BY ProductName
;
"""
pd.read_sql(query,db).head()

,ProductID,ProductName,SupplierID,CategoryID,Unit,Price
0,17,Alice Mutton,7,6,20 - 1 kg tins,39.0
1,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.0
2,40,Boston Crab Meat,19,8,24 - 4 oz tins,18.4
3,60,Camembert Pierrot,28,4,15 - 300 g rounds,34.0
4,18,Carnarvon Tigers,7,8,16 kg pkg.,62.5


In [45]:
query = """
SELECT *
FROM Products
    ORDER BY ProductName DESC
;
"""
pd.read_sql(query,db).head()

,ProductID,ProductName,SupplierID,CategoryID,Unit,Price
0,47,Zaanse koeken,22,3,10 - 4 oz boxes,9.50
1,64,Wimmers gute Semmelknödel,12,5,20 bags x 4 pieces,33.25
2,63,Vegie-spread,7,2,15 - 625 g jars,43.90
3,50,Valkoinen suklaa,23,3,12 - 100 g bars,16.25
4,7,Uncle Bob's Organic Dried Pears,3,7,12 - 1 lb pkgs.,30.00


In [89]:
query="""
SELECT name, sql
From sqlite_master
;
"""
pd.read_sql(query,db)

,name,sql
0,Categories,"CREATE TABLE Categories(\n ""CategoryID"" TEXT,\n ""CategoryName"" TEXT,\n ""Description"" TEXT\n)"
1,Customers,"CREATE TABLE Customers(\n ""CustomerID"" TEXT,\n ""CustomerName"" TEXT,\n ""ContactName"" TEXT,\n ""Address"" TEXT,\n ""City"" TEXT,\n ""PostalCode"" TEXT,\n ""Country"" TEXT\n)"
2,Employees,"CREATE TABLE Employees(\n ""EmployeeID"" TEXT,\n ""LastName"" TEXT,\n ""FirstName"" TEXT,\n ""BirthDate"" TEXT,\n ""Photo"" TEXT,\n ""Notes"" TEXT\n)"
3,Shippers,"CREATE TABLE Shippers(\n ""ShipperID"" TEXT,\n ""ShipperName"" TEXT,\n ""Phone"" TEXT\n)"
4,Suppliers,"CREATE TABLE Suppliers(\n ""SupplierID"" TEXT,\n ""SupplierName"" TEXT,\n ""ContactName"" TEXT,\n ""Address"" TEXT,\n ""City"" TEXT,\n ""PostalCode"" TEXT,\n ""Country"" TEXT,\n ""Phone"" TEXT\n)"
5,Orders,"CREATE TABLE Orders(\nOrderID integer,\nCustomerID integer,\nEmployeeID integer,\nOrderDate date,\nShipperID integer)"
6,OrderDetails,"CREATE TABLE OrderDetails(\nOrderDetailID integer,\nOrderID integer,\nProductID integer,\nQuantity integer)"
7,Products,"CREATE TABLE Products(\nProductID integer,\nProductName text,\nSupplierID integer,\nCategoryID integer,\nUnit text,\nPrice integer\n)"


In [55]:
query = """
SELECT *
FROM OrderDetails
;
"""
orderDetails_df = pd.read_sql(query,db)

In [92]:
orderDetails_df.merge(products_df, on="ProductID").head()

,OrderDetailID,OrderID,ProductID,Quantity,ProductName,SupplierID,CategoryID,Unit,Price
0,1,10248,11,12,Queso Cabrales,5,4,1 kg pkg.,21.0
1,130,10296,11,12,Queso Cabrales,5,4,1 kg pkg.,21.0
2,211,10327,11,50,Queso Cabrales,5,4,1 kg pkg.,21.0
3,281,10353,11,12,Queso Cabrales,5,4,1 kg pkg.,21.0
4,314,10365,11,24,Queso Cabrales,5,4,1 kg pkg.,21.0


In [ ]:
# 1월 12일까지 마무리하기(+발표)
# 동기,취지,의도
# 주제
# 방법

In [84]:
pd.read_sql("SELECT * FROM Orders;", db).nunique()

OrderID       196
CustomerID     74
EmployeeID      9
OrderDate     160
ShipperID       3
dtype: int64

In [85]:
pd.read_sql("SELECT * FROM Customers;", db).nunique()

CustomerID      91
CustomerName    91
ContactName     91
Address         91
City            69
PostalCode      87
Country         21
dtype: int64

In [62]:
query="""
SELECT *
FROM
    Customers, Orders
;
"""
len(pd.read_sql(query,db))

17836

In [73]:
query="""
SELECT C.CustomerID, C.CustomerName, O.CustomerID, O.OrderID
FROM 
    Orders O, Customers C
WHERE
    C.CustomerID = O.CustomerID
;
"""
pd.read_sql(query,db).head()

,CustomerID,CustomerName,CustomerID,OrderID
0,2,Ana Trujillo Emparedados y helados,2,10308
1,3,Antonio Moreno Taquería,3,10365
2,4,Around the Horn,4,10355
3,4,Around the Horn,4,10383
4,5,Berglunds snabbköp,5,10278


In [86]:
query="""
SELECT C.CustomerID, C.CustomerName, O.CustomerID, O.OrderID
FROM Customers C
    JOIN
        Orders O
    ON
        C.CustomerID = O.CustomerID
;
"""
len(pd.read_sql(query,db))

196

In [79]:
query="""
SELECT C.CustomerID, C.CustomerName, O.CustomerID, O.OrderID
FROM Customers C
    JOIN
        Orders O
;
"""
len(pd.read_sql(query,db))

17836

In [ ]:
# 스크래핑 결과를 디비에 밀어넣기
for item in results:
    query="""
    INSERT INTO movies
    VALUES({id},{name},{url})
    ;
    """.format(id=item[0],name=item[1],url=item[2])
    pd.read_sql(query,db)

In [96]:
query = """
SELECT sum(Price)
FROM
    Products;
"""
pd.read_sql(query,db)

,sum(Price)
0,2222.71


In [97]:
query = """
SELECT *
FROM
    Orders
;
"""
orders_df = pd.read_sql(query, db)
orders_df.head()

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
0,10248,90,5,1996-07-04,3
1,10249,81,6,1996-07-05,1
2,10250,34,4,1996-07-08,2
3,10251,84,3,1996-07-08,1
4,10252,76,4,1996-07-09,2


In [99]:
date_group = orders_df.groupby("OrderDate")

In [109]:
date_group.get_group("1996-07-08")

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
2,10250,34,4,1996-07-08,2
3,10251,84,3,1996-07-08,1


In [107]:
orders_df["OrderDate"].unique()

array(['1996-07-04', '1996-07-05', '1996-07-08', '1996-07-09',
       '1996-07-10', '1996-07-11', '1996-07-12', '1996-07-15',
       '1996-07-16', '1996-07-17', '1996-07-18', '1996-07-19',
       '1996-07-22', '1996-07-23', '1996-07-24', '1996-07-25',
       '1996-07-26', '1996-07-29', '1996-07-30', '1996-07-31',
       '1996-08-01', '1996-08-02', '1996-08-05', '1996-08-06',
       '1996-08-07', '1996-08-08', '1996-08-09', '1996-08-12',
       '1996-08-13', '1996-08-14', '1996-08-15', '1996-08-16',
       '1996-08-19', '1996-08-20', '1996-08-21', '1996-08-22',
       '1996-08-23', '1996-08-26', '1996-08-27', '1996-08-28',
       '1996-08-29', '1996-08-30', '1996-09-02', '1996-09-03',
       '1996-09-04', '1996-09-05', '1996-09-06', '1996-09-09',
       '1996-09-10', '1996-09-11', '1996-09-12', '1996-09-13',
       '1996-09-16', '1996-09-17', '1996-09-18', '1996-09-19',
       '1996-09-20', '1996-09-23', '1996-09-24', '1996-09-25',
       '1996-09-26', '1996-09-27', '1996-09-30', '1996-

In [111]:
daily_order_count = pd.DataFrame([
    {
        "OrderDate":OrderDate,
        "Count": len(date_group.get_group(OrderDate))
    } for OrderDate in orders_df["OrderDate"].unique()
    
])
len(daily_order_count)

160

In [122]:
query = """
SELECT OrderDate, count(*) "Count"
FROM
    Orders
    GROUP BY
        OrderDate
;
"""

pd.read_sql(query,db).head()

,OrderDate,Count
0,1996-07-04,1
1,1996-07-05,1
2,1996-07-08,2
3,1996-07-09,1
4,1996-07-10,1
